# Machine Learning

In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch

file = '../data/balanced_data.csv'

data = pd.read_csv(file)
data

,Unnamed: 0,index,Access Gained,Attack Origin,Authentication Required,Availability,CVE ID,CVE Page,CWE ID,Complexity,...,parentID,patch,project,project_after,project_before,target,vul_func_with_fix,processed_func,flaw_line,flaw_line_index
0,0,893,User,Remote,Not required,Partial,CVE-2009-3605,https://www.cvedetails.com/cve/CVE-2009-3605/,CWE-189,Medium,...,0131f0a01cba8691d10a18de1137a4744988b346,"@@ -16,7 +16,7 @@\n //\n // Copyright (C) 2005...",poppler,https://cgit.freedesktop.org/poppler/poppler/t...,https://cgit.freedesktop.org/poppler/poppler/t...,0,void CairoOutputDev::drawImageMaskRegular(GfxS...,void CairoOutputDev::drawImageMaskRegular(GfxS...,NaN,NaN
1,1,78269,None,Local,Not required,Partial,CVE-2018-16427,https://www.cvedetails.com/cve/CVE-2018-16427/,CWE-125,Low,...,NaN,"@@ -1467,7 +1467,7 @@ coolkey_find_attribute(s...",OpenSC,8fe377e93b4b56060e5bbfb6f3142ceaeca744fa,f66ceab4bbc3cae529a22b5f3688076cc119bd7c,0,coolkey_add_object(coolkey_private_data_t *pri...,coolkey_add_object(coolkey_private_data_t *pri...,NaN,NaN
2,2,83297,None,Remote,Not required,Partial,CVE-2018-10017,https://www.cvedetails.com/cve/CVE-2018-10017/,CWE-125,Medium,...,NaN,"@@ -1204,7 +1204,8 @@ std::vector<GetLengthTyp...",openmpt,492022c7297ede682161d9c0ec2de15526424e76,0edf007f58b23ebe9ffaad2c209fc29fa8ce2586,0,void CSoundFile::FineVolumeUp(ModChannel *pChn...,void CSoundFile::FineVolumeUp(ModChannel *pChn...,NaN,NaN
3,3,139641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"@@ -67,6 +67,7 @@\n #include ""modules/webaudio...",Chrome,6834289784ed45b5524de0fb7ef43ae283b0d6d3,c5fd7422d2c218c47910043e0a7298801c85d571,0,OscillatorNode* AudioContext::createOscillator...,OscillatorNode* AudioContext::createOscillator...,NaN,NaN
4,4,128343,None,Remote,Not required,Partial,CVE-2014-3191,https://www.cvedetails.com/cve/CVE-2014-3191/,CWE-416,Low,...,NaN,"@@ -118,6 +118,7 @@ FrameView::FrameView(Local...",Chrome,11a4cc4a6d6e665d9a118fada4b7c658d6f70d95,09298a5df529dfb55d846f5f85df4df437fac052,0,void FrameView::setMediaType(const AtomicStrin...,void FrameView::setMediaType(const AtomicStrin...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21795,10895,186741,None,Remote,Not required,Partial,CVE-2018-6111,https://www.cvedetails.com/cve/CVE-2018-6111/,CWE-20,Medium,...,NaN,"@@ -148,7 +148,7 @@ std::vector<PageHandler*> ...",Chrome,3c8e4852477d5b1e2da877808c998dc57db9460f,488e9ef26e7e265f627b3db349ad4440b4575e93,1,void PageHandler::SetRenderer(RenderProcessHos...,void PageHandler::SetRenderer(RenderProcessHos...,NaN,NaN
21796,10896,182754,None,Remote,Not required,Partial,CVE-2019-14459,https://www.cvedetails.com/cve/CVE-2019-14459/,CWE-190,Low,...,NaN,"@@ -1247,7 +1247,7 @@ int i;\n \t\tuint32_t ta...",nfdump,3b006ededaf351f1723aea6c727c9edd1b1fff9b,7c0d06afdf8cd3ecd57d21f714f7f683083875a2,1,static void Process_ipfix_template_add(exporte...,static void Process_ipfix_template_add(exporte...,\t\tif ( size_left && size_left < 4 ) {,11
21797,10897,187784,None,Remote,Not required,Complete,CVE-2016-2508,https://www.cvedetails.com/cve/CVE-2016-2508/,CWE-119,Medium,...,NaN,"@@ -211,6 +211,9 @@\n\n \n for (size_t i =...",Android,https://android.googlesource.com/platform/fram...,https://android.googlesource.com/platform/fram...,1,status_t NuPlayer::GenericSource::initFromData...,status_t NuPlayer::GenericSource::initFromData...,if (track != NULL) {/~/ mSo...,"121,122,123,124,125,126,127,128,129,130,131,13..."
21798,10898,183686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"@@ -18,8 +18,6 @@\n #include ""chrome/browser/s...",Chrome,19190765882e272a6a2162c89acdb29110f7e3cf,603b62a3200ccc75370d3750b5a76f8f8960d299,1,void MigrationTest::SetUpVersion76Database() {...,void MigrationTest::SetUpVersion76Database() {...,sql::Connection connection;/~/ ASSERT_TRUE(...,"1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."


## Features

In [2]:
feature_columns = ['processed_func']
target_columns = ['target']

target = data[target_columns]
features = data[feature_columns]
features

,processed_func
0,void CairoOutputDev::drawImageMaskRegular(GfxS...
1,coolkey_add_object(coolkey_private_data_t *pri...
2,void CSoundFile::FineVolumeUp(ModChannel *pChn...
3,OscillatorNode* AudioContext::createOscillator...
4,void FrameView::setMediaType(const AtomicStrin...
...,...
21795,void PageHandler::SetRenderer(RenderProcessHos...
21796,static void Process_ipfix_template_add(exporte...
21797,status_t NuPlayer::GenericSource::initFromData...
21798,void MigrationTest::SetUpVersion76Database() {...


In [3]:
target

,target
0,0
1,0
2,0
3,0
4,0
...,...
21795,1
21796,1
21797,1
21798,1


In [4]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
nl_tokens=tokenizer.tokenize("return maximum value")

In [6]:
nl_tokens

features['function_tokenized'] = features["processed_func"].apply(tokenizer.tokenize)
#features['function_tokenized'] = tokenizer.encode(features["processed_func"], truncation=True, max_length=1)
features

Token indices sequence length is longer than the specified maximum sequence length for this model (867 > 512). Running this sequence through the model will result in indexing errors
C:\Users\kevwil\AppData\Local\Temp\ipykernel_23252\2289423554.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['function_tokenized'] = features["processed_func"].apply(tokenizer.tokenize)


,processed_func,function_tokenized
0,void CairoOutputDev::drawImageMaskRegular(GfxS...,"[void, ĠCairo, Output, Dev, ::, draw, Image, M..."
1,coolkey_add_object(coolkey_private_data_t *pri...,"[cool, key, _, add, _, object, (, cool, key, _..."
2,void CSoundFile::FineVolumeUp(ModChannel *pChn...,"[void, ĠCS, ound, File, ::, Fine, Volume, Up, ..."
3,OscillatorNode* AudioContext::createOscillator...,"[O, scill, ator, Node, *, ĠAudio, Context, ::,..."
4,void FrameView::setMediaType(const AtomicStrin...,"[void, ĠFrame, View, ::, set, Media, Type, (, ..."
...,...,...
21795,void PageHandler::SetRenderer(RenderProcessHos...,"[void, ĠPage, Handler, ::, Set, R, end, erer, ..."
21796,static void Process_ipfix_template_add(exporte...,"[static, Ġvoid, ĠProcess, _, ip, fix, _, templ..."
21797,status_t NuPlayer::GenericSource::initFromData...,"[status, _, t, ĠNu, Player, ::, Generic, Sourc..."
21798,void MigrationTest::SetUpVersion76Database() {...,"[void, ĠMigration, Test, ::, Set, Up, Version,..."


In [7]:
tokens_ids=features["function_tokenized"].apply(tokenizer.convert_tokens_to_ids)
tokens_ids

0        [47908, 14794, 48293, 30504, 38304, 24686, 853...
1        [24336, 5282, 1215, 4917, 1215, 40412, 1640, 2...
2        [47908, 7038, 9834, 9966, 38304, 45089, 39781,...
3        [673, 46003, 2630, 48271, 3226, 20555, 48522, ...
4        [47908, 37280, 22130, 38304, 8738, 18801, 4011...
                               ...                        
21795    [47908, 7086, 49191, 38304, 28512, 500, 1397, ...
21796    [42653, 13842, 19149, 1215, 1588, 23032, 1215,...
21797    [29552, 1215, 90, 12907, 35605, 38304, 49144, ...
21798    [47908, 26742, 34603, 38304, 28512, 10926, 473...
21799    [330, 20815, 245, 1215, 571, 7485, 1215, 45705...
Name: function_tokenized, Length: 21800, dtype: object

In [ ]:
model = AutoModel.from_pretrained("microsoft/codebert-base")
context_embeddings=model(torch.tensor(tokens_ids)[None,:])[0]

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

In [ ]:
from sklearn.ensemble import RandomForestClassifier

feature_names = [f"feature {i}" for i in range(X.shape[1])]
forest = RandomForestClassifier(random_state=0)
forest.fit(X_train, y_train)



In [ ]:
X = df.drop(['target'], axis=1)
y = df["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
model_list = ['LR', 'SVM', 'KNN', 'Decision Tree', 'Random Forest', 'Naive Bayes']

acc_list = []
auc_list = []
cm_list = []

for model in model_pipeline:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc_list.append(metrics.accuracy_score(y_test, y_pred))
    fpr, tpr, _thresholds = metrics.roc_curve(y_test, y_pred)
    auc_list.append(round(metrics.auc(fpr, tpr), 2))
    cm_list.append(confusion_matrix(y_test, y_pred))

In [ ]:
fig = plt.figure(figsize=(10, 10))
for i in range(len(model_list)):
    ax = fig.add_subplot(3, 2, i+1)
    ax.set_title(model_list[i])
    sns.heatmap(cm_list[i], annot=True, fmt='d', ax=ax)
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Actual')